<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2018-1, 2018-2, 2019-1 y 2019-2.</font>
</p>

## Diseño de software: *Front-end* y *Back-end*

En la ingeniería de software existen los conceptos de ***front-end*** y ***back-end*** para referirse a la separación que existe entre la capa de presentación y la capa de acceso a los datos, respectivamente. 

En el caso de interfaces gráficas:

- el ***front-end*** está relacionado a la **interfaz gráfica** con la cual el usuario interactúa, y 
- el ***back-end*** se refiere a la **lógica** detrás de ella. 

Esta separación se alinea con un principio importante en el diseño de *software* de calidad que indica que siempre debemos buscar **alta cohesión y bajo acomplamiento** en nuestros diseños.

- **Cohesión**: Cada una de las componentes del software debe realizar **solo las tareas para las cuales fue creada**, delegando otras tareas a otras componentes según corresponda. Por ejemplo, si tengo una clase `SimulaciónDeParque`, un diseño altamente cohesionado incluiría métodos como `iniciar_simulación()` o `detener_simulación()`, pero no métodos como `limpiar_atracción()` o `ingresar_clientes_a_restaurant()`, ya que la clase `SimulaciónDeParque` fue diseñada para administrar la simulación y no para hacerse cargo de métodos que deberían ser ejecutados (*delegados a*) otras clases de la simulación.
- **Acoplamiento**: Cuando la modificación de una componente implica que es necesario modificar otra componente para que la implementación del cambio sea correcta y completa. Por ejemplo, si al modificar los atributos de una clase A, también se deben modificar los atributos de otra clase B, se dice que hay *alto acoplamiento* entre las clases A y B. Un buen diseño intenta reducir el acoplamiento entre clases.

De ahí que siempre debemos buscar **ALTA COHESIÓN y BAJO ACOPLAMIENTO** en nuestros diseños.

En el caso de la separación del ***front-end*** y ***back-end***, algunas ventajas son las siguientes:

1. **Modularidad**: Permite cambiar cualquiera de las dos partes sin afectar la otra (bajo acoplamiento). En particular, podemos editar el *front-end* suponiendo que las funciones utilizadas por el *back-end* mantienen su comportamiento. Al mismo tiempo, es posible modularizar el *back-end* de manera independiente del *front-end*. Podemos reescribir el código para hacerlo cada vez más eficiente y específico (alta cohesión). Podemos incluso modularizar el *back-end* en muchos archivos distintos y luego consultar todas las funcionalidades desde un solo archivo de conexión con el *front-end* (alta cohesión).
1. **Uso de recursos**: Algunas veces el *front-end* está corriendo en un computador distinto al *back-end*. Si el *back-end* ejecuta cálculos muy costosos, no nos gustaría cargarle este tiempo computacional a la interfaz gráfica. Un ejemplo claro de esto son los navegadores de internet (*browsers*), donde la mayoría de los cálculos o datos que queremos obtener se generan en un servidor de *back-end* y el resultado obtenido solo se muestra gráficamente en el computador del usuario (*front-end*). De este modo, nuestro computador no tiene que sobrecargarse procesando cosas.
1. **Escalamiento**: Por un lado, permite hacer crecer un *software* sin mucha interferencia a las funcionalidades antiguas. Por otro lado, permite distribuir el procesamiento en el *back-end* utilizando múltiples réplicas de este, lo que es muy usado en la web.
1. **Experticia**: Usualmente los desarrollador de un *front-end* tienen un tipo de experiencia muy distinta a la que tienen quienes desarrollan el *back-end*. Mantenerlos separados permite obtener lo mejor de ambas partes.
1. **Mantención**: Es posible hacer *testing* parte por parte de una pieza de software, e introducir correcciones o mejoras evitando que un alto acoplamiento de funcionalidades haga que las modificaciones deban ser propagadas a múltiples partes del código.
1. **Evolución del *software*/versionamiento**: Si quieres cambiar completamente una de las partes puedes hacerlo sin problema, mientras las funciones utilizadas en el *front-end* sigan teniendo los mismos nombres que antes (dicho de otra forma, mientras se mantenga la misma interfaz de métodos). De esta manera, por ejemplo, si programas un *back-end* para PyQt5 y luego quieres usarlo para PyQt6, puedes hacerlo sin problemas (alta cohesión). O mejor aún, exportar tus funcionalidades para un *back-end* web.

Estas ventajas pueden transformarse en costos si es que hay que tener dos equipos distintos de desarrollo o hay que mantener dos *software* distintos. Sin embargo, las ventajas siguen siendo mayores.

### Ejemplo: Escribiendo un programa separando *front-end* y *back-end*

Se mostrará mediante ejemplos la evolución de un programa que inicialmente no sigue la arquitectura *front-end*/*back-end*, y termina siguendola. El programa que se construye tiene por objetivo recibir una lista de números (separados por coma), y muestra como resultado la lista ordenada de números.

#### Versión 1: Programa poco cohesivo

El siguiente programa resuelve lo pedido, y sigue el siguiente flujo:

- Define los componentes de interfaz gráfica: dos *labels*, un *input* de texto y un botón.
- Conecta el clic sobre el botón con la funcionalidad de ordenamiento.
- Al hacer clic y gatillar el evento de ordenar:
    - Por simplicidad, se remueven todos los espacios de `texto_input`
    - Se verifica que el texto recibido (la lista de números ingresada por el usuario) sea válido
    - Si no es válido, se notifica al usuario mediante una etiqueta de texto.
    - Si lo es, se ordenan los números mediante un algoritmo que saca el mínimo sucesivamente y luego muestra el resultado.

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QPushButton, QLineEdit)

class Ventana(QWidget):

    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):

        self.etiqueta = QLabel('Ingresa una lista de números separados por comas:', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.input = QLineEdit('', self)
        self.input.setGeometry(20, 40, 360, 20)

        self.boton = QPushButton('Ordenar', self)
        self.boton.setGeometry(20, 70, 360, 30)
        self.boton.clicked.connect(self.ordenar)

        self.resultado = QLabel('', self)
        self.resultado.move(20, 100)
        self.resultado.resize(self.resultado.sizeHint())

        self.setGeometry(700, 300, 400, 150)
        self.setWindowTitle('Ordenador de números')
        self.show()

    def ordenar(self):
        texto_input = self.input.text()
        texto_input = texto_input.replace(' ', '')
        caracteres_posibles = list(map(str, range(0, 10))) + [',']
        for caracter in texto_input:
            if caracter not in caracteres_posibles:
                self.resultado.setText('Input no válido')
                self.resultado.resize(self.resultado.sizeHint())
                return
        lista_de_numeros = [int(porcion) for porcion in texto_input.split(',')]
        numeros_ordenados = []
        while len(lista_de_numeros) > 0:
            minimo_actual = lista_de_numeros[0]
            for numero in lista_de_numeros:
                if numero < minimo_actual:
                    minimo_actual = numero
            numeros_ordenados.append(minimo_actual)
            lista_de_numeros.remove(minimo_actual)
        texto_resultado = ", ".join([str(numero) for numero in numeros_ordenados])
        self.resultado.setText(texto_resultado)
        self.resultado.resize(self.resultado.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    ventana = Ventana()
    sys.exit(app.exec_())

Si bien el código logra lo esperado, notamos primero que es muy poco **cohesivo**. La clase `Ventana` realiza **todas** las operaciones necesarias para lograr el resultado del programa. La clase `Ventana` al ser un componente visual, solo debiese encargarse de tareas visuales que involucren a esta ventana. El método `ordenar` específicamente realiza tareas de procesamiento de datos y tareas visuales en la ventana. Las responsabilidades de tareas no se reparten correctamente entre más componentes, por lo que resulta en código poco cohesivo.

#### Versión 2: Programa cohesivo pero acoplado

En la siguiente versión del programa se logra separar las responsabilidades de código en distintos componentes lógicos. Se crea un módulo llamado `procesamiento.py` que se encarga de la validación del *input* y ordenamiento del la lista de números.

In [ ]:
import sys
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QPushButton, QLineEdit)
from procesamiento import procesar_input

class Ventana(QWidget):

    def __init__(self):
        super().__init__()
        self.inicializa_gui()

    def inicializa_gui(self):

        self.etiqueta = QLabel('Ingresa una lista de números separados por comas:', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.input = QLineEdit('', self)
        self.input.setGeometry(20, 40, 360, 20)

        self.boton = QPushButton('Ordenar', self)
        self.boton.setGeometry(20, 70, 360, 30)
        self.boton.clicked.connect(self.boton_clickeado)

        self.resultado = QLabel('', self)
        self.resultado.move(20, 100)
        self.resultado.resize(self.resultado.sizeHint())

        self.setGeometry(700, 300, 400, 150)
        self.setWindowTitle('Ordenador de números')
        self.show()

    def boton_clickeado(self):
        texto_input = self.input.text()
        texto_resultado = procesar_input(texto_input)
        self.resultado.setText(texto_resultado)
        self.resultado.resize(self.resultado.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    ventana = Ventana()
    sys.exit(app.exec_())

In [ ]:
# procesamiento.py

def es_valido(texto):
    caracteres_posibles = list(map(str, range(0, 10))) + [',']
    for caracter in texto:
        if caracter not in caracteres_posibles:
            return False
    return True

def procesar_input(texto_input):
    texto_input = texto_input.replace(' ', '')
    if not es_valido(texto_input):
        return 'Input no válido'
    lista_de_numeros = [int(porcion) for porcion in texto_input.split(',')]
    numeros_ordenados = []
    while len(lista_de_numeros) > 0:
        minimo_actual = lista_de_numeros[0]
        for numero in lista_de_numeros:
            if numero < minimo_actual:
                minimo_actual = numero
        numeros_ordenados.append(minimo_actual)
        lista_de_numeros.remove(minimo_actual)
    texto_resultado = ", ".join([str(numero) for numero in numeros_ordenados])
    return texto_resultado

Notamos dos beneficios:

- Todo el código encargado de procesamiento de datos queda en un archivo separado y ordenado.
- El código de la ventana principal se simplifica y solo se encarga de tareas visuales.

Es decir, hemos logrado mayor cohesión en nuestro programa.

También, notamos que esta separación ya califica para considerarse arquitectura *front-end*/*back-end*: La clase `Ventana` es parte del *front-end* de nuestro programa al encargarse de solo tareas visuales y de interfaz, mientras que `procesamiento.py` es parte del *back-end* de nuestro programa al encargarse de lógica del programa.

Ahora, ¿es el código resultante poco acoplado? Lamentablemente, no tanto. Esto debido a que la componente visual llama directamente a la componente de lógica (específicamente, la función `procesar_input`). Luego, si eventuamente se modifica el nombre de esta función en el módulo `procesamiento.py`, implica necesarimente cambiar su nombre también en la clase `Ventana` y en cualquier otro lugar que sea referenciado. Puede que este ejemplo parezca simple y poco grave ya que es solo una función la que se importa y utiliza, pero si esto se extrapola a un programa de mayor tamaño con múltiples funciones importadas, se vuelve un mayor problema. Es decir, nuestro programa está relativamente **acoplado**.

Pero, ¿cómo se soluciona esta situación? **Hay** que llamar de una forma u otra a esta función para lograr el objetivo resultante, ¿cómo se logra sin llamar a la función directamente? ¡Señales! 

#### Versión 3:  Programa cohesivo y poco acoplado

En esta última versión, se logra una separación de componentes *front-end*/*back-end* que logra un programa altamente cohesivo, y mediante comunicación via señales se logra un programa levemente acoplado.

La diferencia con la última versión es que se elimina el llamado directo de objetos de distintos componentes y se delega todo a señales para comunicarse. Se crean dos señales: `senal_actualizar` cuyo objetivo es comunicar desde *back-end* al *front-end* una actualización de ventana, y `senal_procesar` cuyo objetivo es comuncar desde *front-end* a *back-end* cuando es necesario procesar *input* nuevo.

Ahora las funciones de procesamiento se encapsulan en una clase `Procesador` para contener la señal de PyQt, y en el código principal es necesario pasar referencias de las señales correspondientes a cada componente. Este último detalle es necesario para que efectivamente se comuniquen los componentes, y es innevitablemente un aspecto de acoplamiento del programa. Es imposible lograr acoplamiento cero en un programa, pero si es posible **minimizarlo**.

In [ ]:
import sys
from PyQt5.QtCore import pyqtSignal
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QPushButton, QLineEdit)
from procesador import Procesador

class Ventana(QWidget):

    senal_actualizar = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.inicializa_gui()
        self.senal_actualizar.connect(self.actualizar_resultado)
        self.senal_procesar = None

    def inicializa_gui(self):

        self.etiqueta = QLabel('Ingresa una lista de números separados por comas:', self)
        self.etiqueta.move(20, 10)
        self.etiqueta.resize(self.etiqueta.sizeHint())

        self.input = QLineEdit('', self)
        self.input.setGeometry(20, 40, 360, 20)

        self.boton = QPushButton('Ordenar', self)
        self.boton.setGeometry(20, 70, 360, 30)
        self.boton.clicked.connect(self.boton_clickeado)

        self.resultado = QLabel('', self)
        self.resultado.move(20, 100)
        self.resultado.resize(self.resultado.sizeHint())

        self.setGeometry(700, 300, 400, 150)
        self.setWindowTitle('Ordenador de números')
        self.show()

    def boton_clickeado(self):
        if self.senal_procesar:
            texto_input = self.input.text()
            self.senal_procesar.emit(texto_input)

    def actualizar_resultado(self, texto):
        self.resultado.setText(texto)
        self.resultado.resize(self.resultado.sizeHint())


if __name__ == '__main__':
    app = QApplication([])
    procesador = Procesador()
    ventana = Ventana()
    procesador.senal_actualizar = ventana.senal_actualizar
    ventana.senal_procesar = procesador.senal_procesar
    sys.exit(app.exec_())

In [ ]:
# procesador.py

from PyQt5.QtCore import QObject, pyqtSignal

class Procesador(QObject):

    senal_procesar = pyqtSignal(str)
    
    def __init__(self):
        super().__init__()
        self.senal_actualizar = None
        self.senal_procesar.connect(self.procesar_input)
    
    def es_valido(self, texto):
        caracteres_posibles = list(map(str, range(0, 10))) + [',']
        for caracter in texto:
            if caracter not in caracteres_posibles:
                return False
        return True

    def procesar_input(self, texto_input):
        texto_input = texto_input.replace(' ', '')
        if not self.es_valido(texto_input):
            self.actualizar_interfaz('Input no válido')
            return
        lista_de_numeros = [int(porcion) for porcion in texto_input.split(',')]
        numeros_ordenados = []
        while len(lista_de_numeros) > 0:
            minimo_actual = lista_de_numeros[0]
            for numero in lista_de_numeros:
                if numero < minimo_actual:
                    minimo_actual = numero
            numeros_ordenados.append(minimo_actual)
            lista_de_numeros.remove(minimo_actual)
        texto_resultado = ", ".join([str(numero) for numero in numeros_ordenados])
        self.actualizar_interfaz(texto_resultado)

    def actualizar_interfaz(self, texto):
        if self.senal_actualizar:
            self.senal_actualizar.emit(texto)

Es importante notar que el código dentro de las dos clases principales creadas (`Ventana` y `Procesador`) nunca llama directamente al código de la otra clase, solo llama y ejecuta código interno. Cada una funciona por si sola y delega la responsabilidad de envio de información a señales.

En general, para escribir un programa de:
- software altamente **cohesivo**, la estrategia es separar de forma adecuada las **responsabilidades** de los distintos compontentes del programa; y escribir distintos módulos y clases destintados a objetivos claros
- software poco **acoplado**, la estrategia es independizar lo más posible distintos compontentes; el uso de **señales** es muy compatible con esta idea ya que reduce el acoplamiento al uso de señales comunes.